<a href="https://colab.research.google.com/github/S-Chaudhuri/Machine-Learning-from-Scratch/blob/main/naivebayes_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import pandas as pd
import numpy as np

In [49]:
df = pd.read_csv("/content/sample_data/Breast cancer data.csv")
df.head()

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,diagnosis
0,17.99,10.38,122.80,1001.0,0.11840,0
1,20.57,17.77,132.90,1326.0,0.08474,0
2,19.69,21.25,130.00,1203.0,0.10960,0
3,11.42,20.38,77.58,386.1,0.14250,0
4,20.29,14.34,135.10,1297.0,0.10030,0


In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   mean_radius      569 non-null    float64
 1   mean_texture     569 non-null    float64
 2   mean_perimeter   569 non-null    float64
 3   mean_area        569 non-null    float64
 4   mean_smoothness  569 non-null    float64
 5   diagnosis        569 non-null    int64  
dtypes: float64(5), int64(1)
memory usage: 26.8 KB


In [51]:
df.describe()

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,diagnosis
count,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,14.127292,19.289649,91.969033,654.889104,0.096360,0.627417
std,3.524049,4.301036,24.298981,351.914129,0.014064,0.483918
min,6.981000,9.710000,43.790000,143.500000,0.052630,0.000000
25%,11.700000,16.170000,75.170000,420.300000,0.086370,0.000000
50%,13.370000,18.840000,86.240000,551.100000,0.095870,1.000000
75%,15.780000,21.800000,104.100000,782.700000,0.105300,1.000000
max,28.110000,39.280000,188.500000,2501.000000,0.163400,1.000000


In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   mean_radius      569 non-null    float64
 1   mean_texture     569 non-null    float64
 2   mean_perimeter   569 non-null    float64
 3   mean_area        569 non-null    float64
 4   mean_smoothness  569 non-null    float64
 5   diagnosis        569 non-null    int64  
dtypes: float64(5), int64(1)
memory usage: 26.8 KB


In [53]:
def calculate_prior(df, Y):
  classes=sorted(list(df[Y].unique()))
  prior=[]
  for i in classes:
    prior.append(len(df[df[Y]]==i)/len(df))
  return prior

In [54]:
def calculate_likelihood_gaussian(df, feat_name, feat_val, Y, label):
  feat = list(df.columns)
  df = df[df[Y]==label]
  mean, std = df[feat_name].mean(),df[feat_name].std()
  p_x_given_y=(1/(np.sqrt(2*np.pi)*std)) * np.exp(-((feat_val-mean)**2 /(2* std**2)))
  mean,std=df[feat_name].mean(), df[feat_name].std()
  p_x_given_y = (1/(np.sqrt(2*np.pi)*std))*np.exp(-((feat_val-mean)**2/(2*std**2)))
  return p_x_given_y

In [55]:
def naive_bayes_gaussian(df,X,Y):
  features = list(df.columns)[:-1]

  prior = calculate_prior(df,Y)
  Y_pred = []
  for x in X:
    labels = sorted(list(df[Y].unique()))
    likelihood = [1]*(len(labels))
    for j in range(len(labels)):
      for i in range(len(features)):
        likelihood[j] *= calculate_likelihood_gaussian(df, features[i], x[i], Y, labels[j])

    post_prob = [1]*len(labels)
    for j in range(len(labels)):
      post_prob[j]*likelihood[j]*prior[j]
    Y_pred.append(np.argmax(post_prob))
  return np.array(Y_pred)


In [56]:
from sklearn.model_selection import train_test_split
df["diagnosis"] = df["diagnosis"].astype(bool)
train,test = train_test_split(df,test_size=.2,random_state=41)
X_test =test.iloc[:,:-1].values
Y_test = test.iloc[:,-1].values
Y_pred = naive_bayes_gaussian(train,X=X_test, Y='diagnosis')

from sklearn.metrics import confusion_matrix, f1_score
print(confusion_matrix(Y_test,Y_pred))
print(f1_score(Y_test,Y_pred))

[[40  0]
 [74  0]]
0.0
